# Visualize statistics on the inflated brain

In [1]:
import os, warnings
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import nibabel as nib
import seaborn as sns

from nilearn import plotting, datasets

warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

model_ids = ['CNorm', 'BNorm']

mpl.rcParams['xtick.labelsize'] = 12 
mpl.rcParams['ytick.labelsize'] = 12 

proj_dir = '/Users/philis/Documents/projects/b-norm_modelling'
os.chdir(proj_dir)

atlas_to_use = 'glasser'

# load the data
df_combined_2yr = pd.read_csv(f'data/{atlas_to_use}/combined_features_2yr.csv')
df_combined_4yr = pd.read_csv(f'data/{atlas_to_use}/combined_features_4yr.csv')

ROI_lbls = df_combined_2yr.columns[15:]

model_out_dir = f'model_output/{atlas_to_use}/'
stats_out_dir = f'stats/{atlas_to_use}'; os.makedirs(stats_out_dir, exist_ok=True)
plot_out_dir = f'plots/{atlas_to_use}'; os.makedirs(plot_out_dir, exist_ok=True)

Index(['lh_avg', 'lh_L_V1', 'lh_L_MST', 'lh_L_V6', 'lh_L_V2', 'lh_L_V3',
       'lh_L_V4', 'lh_L_V8', 'lh_L_4', 'lh_L_3b',
       ...
       'rh_R_p47r', 'rh_R_TGv', 'rh_R_MBelt', 'rh_R_LBelt', 'rh_R_A4',
       'rh_R_STSva', 'rh_R_TE1m', 'rh_R_PI', 'rh_R_a32pr', 'rh_R_p24'],
      dtype='object', length=362)


## helper function

In [2]:
def plot_on_surface(data, stat, timepoint, vmin, vmax, views=['lateral', 'medial'], plot_out_dir='plots/glasser',
                    atlas_to_use='glasser', cmap='coolwarm', use_sym_cbar=True):
    # get the parcellations
    hcp_file_lh = "T1stats/HCP/lh.HCPMMP1.annot"
    parcellation_lh_orig, ctab, names = nib.freesurfer.read_annot(hcp_file_lh)

    hcp_file_rh = "T1stats/HCP/rh.HCPMMP1.annot"
    parcellation_rh_orig, ctab, names = nib.freesurfer.read_annot(hcp_file_rh)

    # Retrieve fsaverage surface dataset for the plotting background. It contains
    # the surface template as pial and inflated version and a sulcal depth maps
    # which is used for shading
    fsaverage = datasets.fetch_surf_fsaverage(mesh="fsaverage")

    parcellation_lh = parcellation_lh_orig.astype(float).copy()
    parcellation_rh = parcellation_rh_orig.astype(float).copy()

    # this_out_dir = os.path.join(plot_out_dir, stat); os.makedirs(this_out_dir, exist_ok=True)
    # print(plot_out_dir)

    if np.abs(vmin) > vmax:
        vmin = vmin
        vmax = np.abs(vmin)
    if stat == 'pvals':
        data.values = -np.log(data.values)
        vmin, vmax = 0, 12
    if stat in ['prctDev']:
        vmin, vmax = 0, vmax

    print("Current stat:", stat, timepoint)
    print(any('negative' in sub for sub in [stat, timepoint]))
    this_cmap = 'winter' if any('negative' in sub for sub in [stat, timepoint]) else cmap

    this_dat = data.values

    for i in range(int(len(ROI_lbls) / 2)):

        if i == 0:
            parcellation_lh[parcellation_lh_orig == i] = np.nan
            parcellation_rh[parcellation_rh_orig == i] = np.nan
        else:
            parcellation_lh[parcellation_lh_orig == i] = this_dat[i]
            parcellation_rh[parcellation_rh_orig == i] = this_dat[i + 181]

    for v, view in enumerate(views):
        fig = plt.figure(figsize=(7, 5))
        out_name = os.path.join(plot_out_dir, f'lh_{timepoint}_{view}.png')
        a = plotting.plot_surf_stat_map(fsaverage['infl_left'], parcellation_lh, hemi='left', figure=fig,
                                        vmin=vmin if stat in ['pvals', 'prctDev'] else None, symmetric_cbar=use_sym_cbar,
                                        vmax=vmax, view=view, alpha=0.7, cmap=this_cmap, colorbar=False,
                                        bg_map=fsaverage['sulc_left'], bg_on_data=True, darkness=.5)
        b = plotting.plot_surf_contours(fsaverage['infl_left'], parcellation_lh_orig, hemi='left', figure=a,
                                        colors=['k' for i in range(181)], output_file=out_name)
        fig.tight_layout()
        plt.close()

        fig = plt.figure(figsize=(7, 5))
        out_name = os.path.join(plot_out_dir, f'rh_{timepoint}_{view}.png')
        a = plotting.plot_surf_stat_map(fsaverage['infl_right'], parcellation_rh, hemi='right', figure=fig,
                                        vmin=vmin if stat in ['pvals', 'prct'] else None, symmetric_cbar=use_sym_cbar,
                                        vmax=vmax, view=view, alpha=0.7, cmap=this_cmap,
                                        colorbar=True if view == 'medial' else False, bg_map=fsaverage['sulc_right'],
                                        bg_on_data=True, darkness=.5)
        b = plotting.plot_surf_contours(fsaverage['infl_right'], parcellation_rh_orig, hemi='right', figure=a,
                                        colors=['k' for i in range(181)], output_file=out_name)
        fig.tight_layout()
        plt.close()


In [3]:
def load_performance_data(model_ids, datasets=['full', 'male', 'female'], timepoints=["2yr", "4yr"], suffix="w4yr", use_bspline=True):
    dfs = []
    bspline_suffix = '_bspline' if use_bspline else ''
    for d, ds in enumerate(datasets):
        for j, mod in enumerate(model_ids):
            for t, tpt in enumerate(timepoints):
                try:
                    if ds == 'full':
                        inter = []
                        for s, sex in enumerate(["male", "female"]):
                            inter.append(pd.read_csv(os.path.join(f'model_output/{atlas_to_use}/', f"{mod}{bspline_suffix}", ds, 
                                                          f"{tpt}_performance_{suffix}.csv")))
                            inter[s]['sex'] = f"full_{sex}"
                        df = pd.concat(inter)
                    else: 
                        df = pd.read_csv(os.path.join(f'model_output/{atlas_to_use}/', f"{mod}{bspline_suffix}", ds, 
                                                      f"{tpt}_performance_{suffix}.csv"))
                        df['sex'] = ds
                    df['timepoint'] = tpt
                    df['dataset'] = ds
                    df['model'] = mod
                    dfs.append(df)
                except:
                    print(f"Warning: {tpt} does not exist for {ds} {mod}")
                    continue
    
    combined = pd.concat(dfs, axis=0).reset_index(drop=True)
    #combined['dRho'] = combined.Rho.sub(combined.groupby(['dataset', 'model', 'timepoint'])['Rho'].transform('mean'))
    return combined

Please note: For some reason some of the RAM is not really freed for a while. At some point it is then freed again, I am not entirely sure when and why.

In [ ]:
stat = 'tstat'
use_sym_cbar = True
use_thresh = False
subs = "f1"
tpt = "2yr"

# we first load all t-scores and the boolean variable whether a given ROI was significantly different
dat = {}
sig = {}
a = ["full", "full_male", "full_female", "male", "female"]
for i, model_data in enumerate(a):
    inter = pd.read_csv(os.path.join(stats_out_dir, 'agesex-vs-thickagesex', f'{model_data}_diff_z_{subs}.csv'), 
                           usecols=["ROI", "t", "pcorr", "tpt"])
    inter = inter[inter.tpt == tpt]
    if i == 0:
        dat["ROI"] = inter.ROI.values
        sig["ROI"] = inter.ROI.values
    dat[model_data] = inter.t.values
    sig[model_data] = inter.pcorr.values

data = pd.DataFrame(dat)
sig = pd.DataFrame(sig)

vmin, vmax = data.iloc[:, 1:].values.min(), data.iloc[:, 1:].values.max()

### no changes necessary below here ###

# get the parcellations
hcp_file_lh = "T1stats/HCP/lh.HCPMMP1.annot"
parcellation_lh_orig, ctab, names  = nib.freesurfer.read_annot(hcp_file_lh)

hcp_file_rh = "T1stats/HCP/rh.HCPMMP1.annot"
parcellation_rh_orig, ctab, names  = nib.freesurfer.read_annot(hcp_file_rh)

cmap = 'coolwarm'

# Retrieve fsaverage surface dataset for the plotting background. It contains
# the surface template as pial and inflated version and a sulcal depth maps
# which is used for shading
fsaverage = datasets.fetch_surf_fsaverage(mesh="fsaverage")

parcellation_lh = parcellation_lh_orig.astype(float).copy()
parcellation_rh = parcellation_rh_orig.astype(float).copy()

if np.abs(vmin) > vmax:
    vmax = np.abs(vmin)

print(vmin, vmax)

for x, col in enumerate(data.columns[1:]):  
    
    for i in range(int(len(ROI_lbls)/2)):
            
        if use_thresh:
            if sig[col][i]:
                parcellation_lh[parcellation_lh_orig == i] = data[col].values[i]
            else:
                parcellation_lh[parcellation_lh_orig == i] = np.nan
                
            if sig[col][i+181]:
                parcellation_rh[parcellation_rh_orig == i] = data[col].values[i+181]
            else:
                parcellation_rh[parcellation_rh_orig == i] = np.nan
        else:
            if i > 0:
                parcellation_lh[parcellation_lh_orig == i] = data[col].values[i]
                parcellation_rh[parcellation_rh_orig == i] = data[col].values[i+181]
    
        if i == 0:
            parcellation_lh[parcellation_lh_orig == i] = np.nan
            parcellation_rh[parcellation_rh_orig == i] = np.nan
    

    this_out_dir = os.path.join(plot_out_dir, 'agesex-vs-thickagesex', col, subs); os.makedirs(this_out_dir, exist_ok=True)
    for v, view in enumerate(['lateral', 'medial']): 
    
        fig = plt.figure(figsize=(7, 5))
        out_name = os.path.join(this_out_dir, f'lh_{view}_{tpt}_thresh.png') if use_thresh else os.path.join(this_out_dir, f'lh_{view}_{tpt}.png')
        a = plotting.plot_surf_stat_map(fsaverage['infl_left'], parcellation_lh, hemi='left', figure=fig, vmin=vmin if stat in ['pvals', 'prct'] else None, symmetric_cbar=use_sym_cbar, 
                                        vmax=vmax, view=view, alpha=0.7, cmap=cmap, colorbar=False, bg_map=fsaverage['sulc_left'], bg_on_data=True, darkness=.5)
        b = plotting.plot_surf_contours(fsaverage['infl_left'], parcellation_lh_orig, hemi='left', figure=a, colors=['k' for i in range(181)], output_file=out_name)
        fig.tight_layout()
        plt.close()
    
        fig = plt.figure(figsize=(7, 5))
        out_name = os.path.join(this_out_dir, f'rh_{view}_{tpt}_thresh.png') if use_thresh else os.path.join(this_out_dir, f'rh_{view}_{tpt}.png')
        a = plotting.plot_surf_stat_map(fsaverage['infl_right'], parcellation_rh, hemi='right', figure=fig, vmin=vmin if stat in ['pvals', 'prct'] else None, symmetric_cbar=use_sym_cbar,
                                        vmax=vmax, view=view, alpha=0.7, cmap=cmap, colorbar=True if view == 'medial' else False, bg_map=fsaverage['sulc_right'], bg_on_data=True, darkness=.5)
        b = plotting.plot_surf_contours(fsaverage['infl_right'], parcellation_rh_orig, hemi='right', figure=a, colors=['k' for i in range(181)], output_file=out_name)
        fig.tight_layout()
        plt.close()

os.system(f'say "done plotting"');

## association with PDSmean

In [17]:
datasplits = ['male', 'female']
stat = 'tstat'

# load the data
data = []
for d, ds in enumerate(datasplits):
    for m, model_id in enumerate(model_ids):
        for t, tpt in enumerate(["2yr", "4yr"]):
            a = pd.read_csv(os.path.join(stats_out_dir, model_id, ds, f'asso_pds-z_{tpt}.csv'), usecols=['ROI', stat, 'pfdr', 'sex', 'model', 'timepoint'])
            data.append(a)

data = pd.concat(data, axis=0)
vmin, vmax = data[stat].values.min(), data[stat].values.max()
print('min:', vmin, 'max:', vmax)

cmap = 'coolwarm'

min: -3.9850811327431472 max: 3.6325181167141407


In [18]:
threshold = False
for d, ds in enumerate(datasplits):
    for m, model_id in enumerate(model_ids):
        for t, tpt in enumerate(["2yr", "4yr"]):
            in_data = data[(data.model == model_id) & (data.sex == ds) & (data.timepoint==tpt)]
            if threshold:
                in_data.loc[in_data.pfdr > 0.05, stat] = np.nan
                save_dir = os.path.join(plot_out_dir, model_id, ds, f"{stat}_thresh")
                os.makedirs(save_dir, exist_ok=True)
            else:
                save_dir = os.path.join(plot_out_dir, model_id, ds, stat)
                os.makedirs(save_dir, exist_ok=True)
            plot_on_surface(in_data[stat], stat, tpt, vmin, vmax, cmap=cmap, plot_out_dir=save_dir)

os.system(f'say "done plotting"');

[get_dataset_dir] Dataset found in /Users/philis/nilearn_data/fsaverage

Current stat: tstat 2yr
False


/var/folders/f9/sy1dpwfx5vx_30_pgdqd5jmm0000gn/T/ipykernel_25658/1846632969.py:52: UserWarning: hemi='left' was passed with type(roi_map)=<class 'numpy.ndarray'> and type(surf_mesh)=<class 'pathlib.PosixPath'>.
This value will be ignored as it is only used when 'roi_map' is a SurfaceImage instance and  / or 'surf_mesh' is a PolyMesh instance.
  b = plotting.plot_surf_contours(fsaverage['infl_left'], parcellation_lh_orig, hemi='left', figure=a,
/var/folders/f9/sy1dpwfx5vx_30_pgdqd5jmm0000gn/T/ipykernel_25658/1846632969.py:54: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/f9/sy1dpwfx5vx_30_pgdqd5jmm0000gn/T/ipykernel_25658/1846632969.py:64: UserWarning: hemi='right' was passed with type(roi_map)=<class 'numpy.ndarray'> and type(surf_mesh)=<class 'pathlib.PosixPath'>.
This value will be ignored as it is only used when 'roi_map' is a SurfaceImage instance and  / or 'surf_mesh' is a PolyMesh

[get_dataset_dir] Dataset found in /Users/philis/nilearn_data/fsaverage

Current stat: tstat 4yr
False


...

Current stat: tstat 4yr
False


[get_dataset_dir] Dataset found in /Users/philis/nilearn_data/fsaverage

Current stat: tstat 2yr
False


[get_dataset_dir] Dataset found in /Users/philis/nilearn_data/fsaverage

Current stat: tstat 4yr
False


In [ ]:
datasplits = ['male', 'female']
stat_val = 'prctDev'
stats = ['positive_2yr', 'negative_2yr', 'positive_4yr', 'negative_4yr']

cols_to_use = ['ROI', 'positive_2yr', 'negative_2yr', 'positive_4yr', 'negative_4yr']

# load the data
data = []
for d, ds in enumerate(datasplits):
    for m, model_id in enumerate(model_ids):
        a = pd.read_csv(os.path.join(stats_out_dir, model_id, ds, f'across-subjects_prct-deviators_thresh-1.96.csv'))
        a['model'] = model_id
        a['sex'] = ds
        data.append(a)

data = pd.concat(data, axis=0)
data_to_use = data[[*cols_to_use, 'model', 'sex']]
vmin, vmax = data_to_use.iloc[:, 1:-2].values.min(), data_to_use.iloc[:, 1:-2].values.max()
print('min:', vmin, 'max:', vmax)

cmap = 'autumn'

In [ ]:
for d, ds in enumerate(datasplits):
    for m, model_id in enumerate(model_ids):
        for s, stat in enumerate(stats):
            in_data = data[(data.model == model_id) & (data.sex == ds)]
            save_dir = os.path.join(plot_out_dir, model_id, ds, stat_val)
            os.makedirs(save_dir, exist_ok=True)
            plot_on_surface(in_data[stat], stat_val, stat, vmin, vmax, cmap=cmap, plot_out_dir=save_dir, use_sym_cbar=False)

os.system(f'say "done plotting"');

# Performance and other statistics

In [25]:
data = load_performance_data(model_ids, datasets=['male', 'female'], timepoints=["2yr", "4yr"], suffix="w4yr", use_bspline=False)
#data = pd.read_csv(os.path.join(stats_out_dir, 'kw_slope.csv'))

#data["logp"] = -np.log(data.pval.values)

print(data)
threshold = False
use_sym_cbar = True

stat = "EV"
#cmap = 'autumn'
cmap = 'coolwarm'

if stat == "logp":
    vmin, vmax = data[~data[stat].isna()][stat].values.min(), np.floor(data[~data[stat].isna()][stat].values.max())
else:
    vmin, vmax = data[~data[stat].isna()][stat].values.min(), data[~data[stat].isna()][stat].values.max()

    
if np.abs(vmin)>vmax:
    vmax = np.abs(vmin)
if stat in ["logp", "pval"]:
    vmin=0

if (vmax > 10) and (stat=="kurtosis"):
    vmax=10

print(vmin, vmax)

# CHANGE BACK

             ROI       MSLL        EV      SMSE      RMSE       Rho  \
0         lh_avg  11.205488  0.030973  0.981000  0.074258  0.188967   
1        lh_L_V1  -0.014291  0.191483  0.808519  0.109796  0.437939   
2       lh_L_MST   0.009943  0.019779  0.980800  0.173800  0.146246   
3        lh_L_V6   0.083594  0.042965  1.020238  0.139618  0.210023   
4        lh_L_V2   0.249516  0.080922  0.920032  0.113726  0.284508   
...          ...        ...       ...       ...       ...       ...   
2891  rh_R_STSva  73.265982  0.646680  0.375777  0.092592  0.804204   
2892   rh_R_TE1m  63.956875  0.738673  0.296734  0.115340  0.859461   
2893     rh_R_PI  27.555086  0.466674  0.534481  0.142764  0.684262   
2894  rh_R_a32pr  53.297545  0.626407  0.373853  0.106278  0.792549   
2895    rh_R_p24  45.610192  0.635744  0.372482  0.127960  0.798805   

              BIC      skew  kurtosis     sex timepoint dataset  model  
0    -6757.555228  0.081562  0.369667    male       2yr    male  CNorm  
1

In [26]:
datasplits = ['male', 'female']
for d, ds in enumerate(datasplits):
    for m, model_id in enumerate(model_ids):
        for t, tpt in enumerate(["2yr", "4yr"]):
            in_data = data[(data.model == model_id) & (data.sex == ds) & (data.timepoint==tpt)]
            if threshold:
                in_data.loc[in_data.pval > 0.05, stat] = np.nan
                save_dir = os.path.join(plot_out_dir, model_id, ds, f"{stat}_thresh")
                os.makedirs(save_dir, exist_ok=True)
            else:
                save_dir = os.path.join(plot_out_dir, model_id, ds, stat)
                os.makedirs(save_dir, exist_ok=True)
            plot_on_surface(in_data[stat], stat, tpt, vmin, vmax, cmap=cmap, plot_out_dir=save_dir, use_sym_cbar=use_sym_cbar)

os.system(f'say "done plotting"');

[get_dataset_dir] Dataset found in /Users/philis/nilearn_data/fsaverage

Current stat: EV 2yr
False


/var/folders/f9/sy1dpwfx5vx_30_pgdqd5jmm0000gn/T/ipykernel_25658/1846632969.py:52: UserWarning: hemi='left' was passed with type(roi_map)=<class 'numpy.ndarray'> and type(surf_mesh)=<class 'pathlib.PosixPath'>.
This value will be ignored as it is only used when 'roi_map' is a SurfaceImage instance and  / or 'surf_mesh' is a PolyMesh instance.
  b = plotting.plot_surf_contours(fsaverage['infl_left'], parcellation_lh_orig, hemi='left', figure=a,
/var/folders/f9/sy1dpwfx5vx_30_pgdqd5jmm0000gn/T/ipykernel_25658/1846632969.py:54: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/f9/sy1dpwfx5vx_30_pgdqd5jmm0000gn/T/ipykernel_25658/1846632969.py:64: UserWarning: hemi='right' was passed with type(roi_map)=<class 'numpy.ndarray'> and type(surf_mesh)=<class 'pathlib.PosixPath'>.
This value will be ignored as it is only used when 'roi_map' is a SurfaceImage instance and  / or 'surf_mesh' is a PolyMesh

[get_dataset_dir] Dataset found in /Users/philis/nilearn_data/fsaverage

Current stat: EV 4yr
False


...

Current stat: EV 4yr
False


[get_dataset_dir] Dataset found in /Users/philis/nilearn_data/fsaverage

Current stat: EV 2yr
False


[get_dataset_dir] Dataset found in /Users/philis/nilearn_data/fsaverage

Current stat: EV 4yr
False


In [ ]:
# if only 1 timepoint
datasplits = ['male', 'female']
for d, ds in enumerate(datasplits):
    for m, model_id in enumerate(model_ids):
        in_data = data[(data.model == model_id) & (data.sex == ds)]
        if threshold:
            in_data.loc[in_data.pval > 0.05, stat] = np.nan
            save_dir = os.path.join(plot_out_dir, model_id, ds, f"{stat}_thresh")
            os.makedirs(save_dir, exist_ok=True)
        else:
            save_dir = os.path.join(plot_out_dir, model_id, ds, stat)
            os.makedirs(save_dir, exist_ok=True)
        plot_on_surface(in_data[stat], stat, 'delta', vmin, vmax, cmap=cmap, plot_out_dir=save_dir, use_sym_cbar=use_sym_cbar)

os.system(f'say "done plotting"');

# Label lobes on brain surface

In [27]:
new_lbls = ROI_lbls.tolist().copy()
new_lbls.remove("lh_avg")
new_lbls.remove("rh_avg")

In [28]:
def load_lobe_data(new_label=None):
    '''
    This function ... TODO
    '''
    
    inter = []
    for hemi in ['LH', 'RH']:
        df = pd.read_csv(f'manuscript/colortab{hemi}_HCP.csv', delimiter=',')
        inter.append(df)

    df_lobe = pd.concat(inter)
    if new_label is not None:
        df_lobe.ROI = new_label

    return df_lobe

lobe_data = load_lobe_data(new_label=new_lbls)
lobe_data

,ROI,num,lobe
0,lh_L_V1,16713023,occipital
1,lh_L_MST,8480566,occipital
2,lh_L_V6,11685438,occipital
3,lh_L_V2,15282711,occipital
4,lh_L_V3,14821391,occipital
...,...,...,...
175,rh_R_STSva,2304049,temporal
176,rh_R_TE1m,3880530,temporal
177,rh_R_PI,4078465,temporal
178,rh_R_a32pr,7164312,frontal


In [29]:
lobe_lbls = lobe_data.lobe.unique()
lobe_data.replace(to_replace=lobe_lbls[:-1], value=list(range(1, len(lobe_lbls[:-1])+1)), inplace=True)

/var/folders/f9/sy1dpwfx5vx_30_pgdqd5jmm0000gn/T/ipykernel_25658/657926383.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lobe_data.replace(to_replace=lobe_lbls[:-1], value=list(range(1, len(lobe_lbls[:-1])+1)), inplace=True)


In [30]:
this_outdir = os.path.join(plot_out_dir, 'lobe_def')
os.makedirs(this_outdir, exist_ok=True)
this_cmap = 'Set2'
views=['lateral', 'medial']

# get the parcellations
hcp_file_lh = "T1stats/HCP/lh.HCPMMP1.annot"
parcellation_lh_orig, ctab, names = nib.freesurfer.read_annot(hcp_file_lh)

hcp_file_rh = "T1stats/HCP/rh.HCPMMP1.annot"
parcellation_rh_orig, ctab, names = nib.freesurfer.read_annot(hcp_file_rh)

# Retrieve fsaverage surface dataset for the plotting background. It contains
# the surface template as pial and inflated version and a sulcal depth maps
# which is used for shading
fsaverage = datasets.fetch_surf_fsaverage(mesh="fsaverage")

parcellation_lh = parcellation_lh_orig.astype(float).copy()
parcellation_rh = parcellation_rh_orig.astype(float).copy()

for i in range(int(len(ROI_lbls) / 2)):

    if i == 0:
        parcellation_lh[parcellation_lh_orig == i] = np.nan
        parcellation_rh[parcellation_rh_orig == i] = np.nan
    else:
        parcellation_lh[parcellation_lh_orig == i] = lobe_data.iloc[i-1, -1]
        parcellation_rh[parcellation_rh_orig == i] = lobe_data.iloc[i-1 + 180, -1]

for v, view in enumerate(views):
    fig = plt.figure(figsize=(7, 5))
    out_name = os.path.join(this_outdir, f'lh_lobes_{view}.png')
    a = plotting.plot_surf_stat_map(fsaverage['infl_left'], parcellation_lh, hemi='left', figure=fig,
                                    vmin=0, symmetric_cbar=False,
                                    vmax=len(lobe_lbls)-1, view=view, alpha=0.7, cmap=this_cmap, colorbar=False,
                                    bg_map=fsaverage['sulc_left'], bg_on_data=True, darkness=.5)
    b = plotting.plot_surf_contours(fsaverage['infl_left'], parcellation_lh_orig, hemi='left', figure=a,
                                    colors=['k' for i in range(181)], output_file=out_name)
    fig.tight_layout()
    plt.close()

    fig = plt.figure(figsize=(7, 5))
    out_name = os.path.join(this_outdir, f'rh_lobes_{view}.png')
    a = plotting.plot_surf_stat_map(fsaverage['infl_right'], parcellation_rh, hemi='right', figure=fig,
                                    vmin=0, symmetric_cbar=False,
                                    vmax=len(lobe_lbls)-1, view=view, alpha=0.7, cmap=this_cmap,
                                    colorbar=True if view == 'medial' else False, bg_map=fsaverage['sulc_right'],
                                    bg_on_data=True, darkness=.5)
    b = plotting.plot_surf_contours(fsaverage['infl_right'], parcellation_rh_orig, hemi='right', figure=a,
                                    colors=['k' for i in range(181)], output_file=out_name)
    fig.tight_layout()
    plt.close()

[get_dataset_dir] Dataset found in /Users/philis/nilearn_data/fsaverage

/var/folders/f9/sy1dpwfx5vx_30_pgdqd5jmm0000gn/T/ipykernel_25658/2206827102.py:37: UserWarning: hemi='left' was passed with type(roi_map)=<class 'numpy.ndarray'> and type(surf_mesh)=<class 'pathlib.PosixPath'>.
This value will be ignored as it is only used when 'roi_map' is a SurfaceImage instance and  / or 'surf_mesh' is a PolyMesh instance.
  b = plotting.plot_surf_contours(fsaverage['infl_left'], parcellation_lh_orig, hemi='left', figure=a,
/var/folders/f9/sy1dpwfx5vx_30_pgdqd5jmm0000gn/T/ipykernel_25658/2206827102.py:39: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/f9/sy1dpwfx5vx_30_pgdqd5jmm0000gn/T/ipykernel_25658/2206827102.py:49: UserWarning: hemi='right' was passed with type(roi_map)=<class 'numpy.ndarray'> and type(surf_mesh)=<class 'pathlib.PosixPath'>.
This value will be ignored as it is only used when 'roi_map' is a SurfaceImage instance and  / or 'surf_mesh' is a PolyMesh